# **P0. Recogida datos estructurados. Crimes & Arrests | LA**

Fuentes de datos:
- Crimes: https://data.lacity.org/Public-Safety/Crime-Data-from-2020-to-Present/2nrs-mtv8/about_data
- Arrests: https://data.lacity.org/Public-Safety/Arrest-Data-from-2020-to-Present/amvf-fr72/about_data

## **Crime LA 2020 - present. csv**

In [14]:
import pandas as pd

In [15]:
df = pd.read_csv("./Crime_Data_from_2020_to_Present.csv")

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 986500 entries, 0 to 986499
Data columns (total 28 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   DR_NO           986500 non-null  int64  
 1   Date Rptd       986500 non-null  object 
 2   DATE OCC        986500 non-null  object 
 3   TIME OCC        986500 non-null  int64  
 4   AREA            986500 non-null  int64  
 5   AREA NAME       986500 non-null  object 
 6   Rpt Dist No     986500 non-null  int64  
 7   Part 1-2        986500 non-null  int64  
 8   Crm Cd          986500 non-null  int64  
 9   Crm Cd Desc     986500 non-null  object 
 10  Mocodes         840065 non-null  object 
 11  Vict Age        986500 non-null  int64  
 12  Vict Sex        846925 non-null  object 
 13  Vict Descent    846914 non-null  object 
 14  Premis Cd       986486 non-null  float64
 15  Premis Desc     985915 non-null  object 
 16  Weapon Used Cd  326368 non-null  float64
 17  Weapon Des

### Preprocesado
Se seleccionan los atributos mas importantes del dataframe. Además, se transforman los tipos de los atributos para que puedan ser insertados en una tabla de postgresql

In [17]:
# Seleccion de las columnas
df = df[['DR_NO','DATE OCC','TIME OCC','AREA','AREA NAME','Crm Cd','Crm Cd Desc','Vict Age', 'Vict Sex','LOCATION','LAT','LON']]

In [18]:
df.head(5)

,DR_NO,DATE OCC,TIME OCC,AREA,AREA NAME,Crm Cd,Crm Cd Desc,Vict Age,Vict Sex,LOCATION,LAT,LON
0,190326475,03/01/2020 12:00:00 AM,2130,7,Wilshire,510,VEHICLE - STOLEN,0,M,1900 S LONGWOOD AV,34.0375,-118.3506
1,200106753,02/08/2020 12:00:00 AM,1800,1,Central,330,BURGLARY FROM VEHICLE,47,M,1000 S FLOWER ST,34.0444,-118.2628
2,200320258,11/04/2020 12:00:00 AM,1700,3,Southwest,480,BIKE - STOLEN,19,X,1400 W 37TH ST,34.0210,-118.3002
3,200907217,03/10/2020 12:00:00 AM,2037,9,Van Nuys,343,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),19,M,14000 RIVERSIDE DR,34.1576,-118.4387
4,220614831,08/17/2020 12:00:00 AM,1200,6,Hollywood,354,THEFT OF IDENTITY,28,M,1900 TRANSIENT,34.0944,-118.3277


In [19]:
# Valores nulos de los atributos
df.isnull().sum()

DR_NO               0
DATE OCC            0
TIME OCC            0
AREA                0
AREA NAME           0
Crm Cd              0
Crm Cd Desc         0
Vict Age            0
Vict Sex       139575
LOCATION            0
LAT                 0
LON                 0
dtype: int64

In [20]:
# Conversion de tipos de datos
df['DR_NO'] = df['DR_NO'].astype('int64')                          # BIGINT
df['DATE OCC'] = pd.to_datetime(df['DATE OCC'], format='%m/%d/%Y %I:%M:%S %p')  # TIMESTAMP
df['TIME OCC'] = df['TIME OCC'].astype('int32')                    # INTEGER
df['AREA'] = df['AREA'].astype('int32')                            # INTEGER
df['AREA NAME'] = df['AREA NAME'].astype('str')                    # VARCHAR(50)
df['Crm Cd'] = df['Crm Cd'].astype('int32')                        # INTEGER
df['Crm Cd Desc'] = df['Crm Cd Desc'].astype('str')                # VARCHAR(255)
df['Vict Age'] = df['Vict Age'].astype('int32')                    # INTEGER
df['Vict Sex'] = df['Vict Sex'].astype('str')                      # CHAR(1)
df['LOCATION'] = df['LOCATION'].astype('str')                      # VARCHAR(255)
df['LAT'] = df['LAT'].astype('float64')                            # DOUBLE PRECISION
df['LON'] = df['LON'].astype('float64')                            # DOUBLE PRECISION

In [21]:
df_procesado = df.copy()

In [22]:
# Se renombran las columnas eliminando los espacios en blanco por _
df_procesado.columns = df_procesado.columns.str.replace(' ', '_')

In [23]:
df_procesado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 986500 entries, 0 to 986499
Data columns (total 12 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   DR_NO        986500 non-null  int64         
 1   DATE_OCC     986500 non-null  datetime64[ns]
 2   TIME_OCC     986500 non-null  int32         
 3   AREA         986500 non-null  int32         
 4   AREA_NAME    986500 non-null  object        
 5   Crm_Cd       986500 non-null  int32         
 6   Crm_Cd_Desc  986500 non-null  object        
 7   Vict_Age     986500 non-null  int32         
 8   Vict_Sex     986500 non-null  object        
 9   LOCATION     986500 non-null  object        
 10  LAT          986500 non-null  float64       
 11  LON          986500 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int32(4), int64(1), object(4)
memory usage: 75.3+ MB


## **Arrests LA 2020 - present. tsv**

In [27]:
df_2 = pd.read_csv('Arrest_Data_from_2020_to_Present_20241020.tsv', sep='\t')

In [28]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 316100 entries, 0 to 316099
Data columns (total 25 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Report ID                 316100 non-null  int64  
 1   Report Type               316100 non-null  object 
 2   Arrest Date               316100 non-null  object 
 3   Time                      316079 non-null  float64
 4   Area ID                   316100 non-null  int64  
 5   Area Name                 316100 non-null  object 
 6   Reporting District        316100 non-null  int64  
 7   Age                       316100 non-null  int64  
 8   Sex Code                  316100 non-null  object 
 9   Descent Code              316100 non-null  object 
 10  Charge Group Code         285867 non-null  float64
 11  Charge Group Description  285819 non-null  object 
 12  Arrest Type Code          316099 non-null  object 
 13  Charge                    316100 non-null  o

### Preprocesado
Se seleccionan los atributos mas importantes del dataframe. Además, se transforman los tipos de los atributos para que puedan ser insertados en una tabla de mariadb

In [29]:
# Seleccion de las columnas
df_2 = df_2[['Report ID','Arrest Date','Area ID','Area Name','Age','Sex Code','Charge', 'Disposition Description','Address','LAT','LON']]

In [30]:
df_2.head(5)

,Report ID,Arrest Date,Area ID,Area Name,Age,Sex Code,Charge,Disposition Description,Address,LAT,LON
0,231413977,07/13/2023 12:00:00 AM,14,Pacific,29,M,25620(A),MISDEMEANOR COMPLAINT FILED,BROOKS AV,33.9908,-118.4765
1,231715213,09/01/2023 12:00:00 AM,17,Devonshire,17,F,45.03(A)LAM,COUNSELED/RELEASED,15800 TULSA ST,34.2682,-118.4760
2,230612568,07/14/2023 12:00:00 AM,6,Hollywood,30,M,647(B)PC,MISDEMEANOR COMPLAINT FILED,WESTERN,34.0845,-118.3091
3,230119897,09/13/2023 12:00:00 AM,1,Central,50,M,41.18A1LAMC,MISDEMEANOR COMPLAINT FILED,5TH,34.0421,-118.2405
4,230214008,07/04/2023 12:00:00 AM,2,Rampart,38,M,41.27(C)LAM,MISDEMEANOR COMPLAINT FILED,BONNIE BRAE,34.0539,-118.2756


In [31]:
# Valores nulos de los atributos
df_2.isnull().sum()

Report ID                      0
Arrest Date                    0
Area ID                        0
Area Name                      0
Age                            0
Sex Code                       0
Charge                         0
Disposition Description    27704
Address                        0
LAT                            0
LON                            0
dtype: int64

In [32]:
# Conversion de tipos de datos
df_2['Report ID'] = df_2['Report ID'].astype('int64')                          # BIGINT
df_2['Arrest Date'] = pd.to_datetime(df_2['Arrest Date'], format='%m/%d/%Y %I:%M:%S %p')  # TIMESTAMP
df_2['Area ID'] = df_2['Area ID'].astype('int32')                              # INTEGER
df_2['Age'] = df_2['Age'].astype('int32')                                      # INTEGER
df_2['Sex Code'] = df_2['Sex Code'].astype('str')                              # CHAR(1)
df_2['Charge'] = df_2['Charge'].astype('str')                                  # VARCHAR(255)
df_2['Disposition Description'] = df_2['Disposition Description'].astype('str')  # VARCHAR(255)
df_2['Address'] = df_2['Address'].astype('str')                                # VARCHAR(255)
df_2['LAT'] = df_2['LAT'].astype('float64')                                    # DOUBLE PRECISION
df_2['LON'] = df_2['LON'].astype('float64')    

In [33]:
df_procesado_2 = df_2.copy()

In [35]:
df_procesado_2.columns = df_procesado_2.columns.str.replace(' ', '_')

In [36]:
df_procesado_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 316100 entries, 0 to 316099
Data columns (total 11 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   Report_ID                316100 non-null  int64         
 1   Arrest_Date              316100 non-null  datetime64[ns]
 2   Area_ID                  316100 non-null  int32         
 3   Area_Name                316100 non-null  object        
 4   Age                      316100 non-null  int32         
 5   Sex_Code                 316100 non-null  object        
 6   Charge                   316100 non-null  object        
 7   Disposition_Description  316100 non-null  object        
 8   Address                  316100 non-null  object        
 9   LAT                      316100 non-null  float64       
 10  LON                      316100 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int32(2), int64(1), object(5)
memory usage: 

## **Creación BD PostgreSQL e Inserción**

In [29]:
!pip install psycopg2-binary --quiet

In [24]:
import psycopg2

In [25]:
try:
    
    ### Conexion a la BD de postgre ###
    
    conn_postgre = psycopg2.connect(
        dbname='postgres',  
        user='hive',        
        password='password',  
        host='hive4-postgres',         
        port='5432'               
    )
    print("Conexión exitosa a PostgreSQL")

    cur = conn_postgre.cursor()
    
   
    ### CREACION DE LA TABLA crimes ###
    #cur.execute(""" DROP TABLE crimes;""")
    cur.execute("""
    CREATE TABLE crimes (
        DR_NO BIGINT PRIMARY KEY,
        DATE_OCC TIMESTAMP,
        TIME_OCC INTEGER,
        AREA_ID INTEGER,
        AREA_NAME VARCHAR(50),
        CRIME_CODE INTEGER,
        CRIME_DESCRIPTION VARCHAR(255),
        VICT_AGE INTEGER,
        GENDER CHAR(3),
        LOCATION VARCHAR(255),
        LATITUDE DOUBLE PRECISION,
        LONGITUDE DOUBLE PRECISION
    );
    """)
    
    print("Tabla creada (si no existía).")

    
    
    ### INSERCIONES ###

    values_to_insert = [
        (row['DR_NO'], row['DATE_OCC'], row['TIME_OCC'], row['AREA'], 
         row['AREA_NAME'], row['Crm_Cd'], row['Crm_Cd_Desc'], 
         row['Vict_Age'], row['Vict_Sex'], row['LOCATION'], 
         row['LAT'], row['LON'])
        for index, row in df_procesado.iterrows()
    ]

    insert_query = """
        INSERT INTO crimes (DR_NO, DATE_OCC, TIME_OCC, AREA_ID, AREA_NAME, CRIME_CODE, 
                            CRIME_DESCRIPTION, VICT_AGE, GENDER, LOCATION, LATITUDE, LONGITUDE)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
    
    cur.executemany(insert_query, values_to_insert)
    conn_postgre.commit()
    
    print(f"Filas insertadas: {len(values_to_insert)}")

    # Verificar y mostrar algunos datos
    cur.execute("SELECT * FROM crimes LIMIT 5;")
    rows = cur.fetchall()
    print("Datos en arrests:", rows)

finally:
    
    if 'cur' in locals():
        cur.close()
    if 'conn_postgre' in locals():
        conn_postgre.close()

Conexión exitosa a PostgreSQL
Tabla creada (si no existía).
Filas insertadas: 986500
Datos en arrests: [(190326475, datetime.datetime(2020, 3, 1, 0, 0), 2130, 7, 'Wilshire', 510, 'VEHICLE - STOLEN', 0, 'M  ', '1900 S  LONGWOOD                     AV', 34.0375, -118.3506), (200106753, datetime.datetime(2020, 2, 8, 0, 0), 1800, 1, 'Central', 330, 'BURGLARY FROM VEHICLE', 47, 'M  ', '1000 S  FLOWER                       ST', 34.0444, -118.2628), (200320258, datetime.datetime(2020, 11, 4, 0, 0), 1700, 3, 'Southwest', 480, 'BIKE - STOLEN', 19, 'X  ', '1400 W  37TH                         ST', 34.021, -118.3002), (200907217, datetime.datetime(2020, 3, 10, 0, 0), 2037, 9, 'Van Nuys', 343, 'SHOPLIFTING-GRAND THEFT ($950.01 & OVER)', 19, 'M  ', '14000    RIVERSIDE                    DR', 34.1576, -118.4387), (220614831, datetime.datetime(2020, 8, 17, 0, 0), 1200, 6, 'Hollywood', 354, 'THEFT OF IDENTITY', 28, 'M  ', '1900    TRANSIENT', 34.0944, -118.3277)]


## **Creación BD MariaDB e Inserción**

In [41]:
!pip install mysql-connector-python --quiet

In [38]:
import mysql.connector

In [39]:
try:
    ### Conexion a la BD de MariaDB ###
    conn_mariadb = mysql.connector.connect(
        user="root",
        password="my_password",
        host="hive4-mariadb",
        database="mariaGESDB"
    )
    print("Conexión exitosa a MariaDB")

    ### CREACION DE LA TABLA arrests ###
    create_table_query = """
    CREATE TABLE IF NOT EXISTS arrests (
        Report_ID BIGINT PRIMARY KEY,
        Arrest_Date DATETIME,
        Area_ID INT,
        Area_Name VARCHAR(50),
        Age INT,
        GENDER CHAR(3),
        Charge VARCHAR(255),
        Disposition_Description VARCHAR(255),
        LOCATION VARCHAR(255),
        LATITUDE DOUBLE PRECISION,
        LONGITUDE DOUBLE PRECISION
    );
    """

    cur = conn_mariadb.cursor()
    
    cur.execute(create_table_query)
    conn_mariadb.commit() 

    print("Tabla creada (si no existía).")

    ### INSERCIONES ###
    
    # Las inserciones se hacen de 1000 en 1000 tuplas
    data_to_insert = []
    for index, row in df_procesado_2.iterrows():
        data_to_insert.append((
            row['Report_ID'], 
            row['Arrest_Date'], 
            row['Area_ID'], 
            row['Area_Name'], 
            row['Age'], 
            row['Sex_Code'], 
            row['Charge'], 
            row['Disposition_Description'], 
            row['Address'], 
            row['LAT'], 
            row['LON']
        ))

    insert_query = """
    INSERT INTO arrests (Report_ID, Arrest_Date, Area_ID, Area_Name, Age, GENDER, 
                         Charge, Disposition_Description, LOCATION, LATITUDE, LONGITUDE)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """

    # Tamaño de lote
    batch_size = 1000 

    # Inserciones en lotes
    for i in range(0, len(data_to_insert), batch_size):
        batch = data_to_insert[i:i + batch_size] 
        try:
            cur.executemany(insert_query, batch)
            conn_mariadb.commit() 
        except mysql.connector.Error as e:
            print(f"Error al insertar el lote {i // batch_size + 1}: {e}")
            conn_mariadb.rollback()

except mysql.connector.Error as err:
    print(f"Error al conectar a MariaDB o al ejecutar una consulta: {err}")
    
finally:
    
    if isinstance(cur, mysql.connector.cursor.MySQLCursor):
        cur.close()
    if isinstance(conn_mariadb, mysql.connector.connection.MySQLConnection):
        conn_mariadb.close()

Conexión exitosa a MariaDB
Tabla creada (si no existía).


In [40]:
### CODIGO QUE SELECCIONA 5 FILAS DE LA TABLA CRIMES ###

cur = conn_mariadb.cursor()
cur.execute("SELECT * FROM arrests LIMIT 5;")
rows = cur.fetchall()

column_names = [i[0] for i in cur.description]

# Número máximo de columnas por línea
max_columns_per_line = 5 

# Función para calcular el ancho de cada columna dinámicamente
def calculate_column_widths(column_names, rows, max_columns_per_line):
    column_widths = []
    
    for i, col_name in enumerate(column_names):
        # Determinar el ancho máximo de cada columna (máximo entre longitud de la cabecera y datos)
        max_data_length = max([len(str(row[i])) for row in rows], default=0)
        column_width = max(len(col_name), max_data_length) + 2  # Añadir 2 para margen
        column_widths.append(column_width)
    
    return column_widths

# Función para imprimir los datos en bloques
def print_columns_in_blocks(column_names, rows, max_columns_per_line):
    num_columns = len(column_names)
    column_widths = calculate_column_widths(column_names, rows, max_columns_per_line)
    
    # Iteracion por bloques de columnas
    for start_idx in range(0, num_columns, max_columns_per_line):
        end_idx = min(start_idx + max_columns_per_line, num_columns)

        # Imprimir las cabeceras para el bloque actual
        header = "".join(f"{column_names[i]:<{column_widths[i]}}" for i in range(start_idx, end_idx))
        print(header)
        print("-" * len(header))  # Separador

        # Imprimir las filas de datos para este bloque
        for row in rows:
            row_data = "".join(f"{str(row[i]):<{column_widths[i]}}" for i in range(start_idx, end_idx))
            print(row_data)

        print("\n")  # Separador entre bloques de columnas

# Llamar a la función para imprimir las columnas en bloques
print_columns_in_blocks(column_names, rows, max_columns_per_line)

Report_ID  Arrest_Date          Area_ID  Area_Name    Age  
-----------------------------------------------------------
1062984    2020-12-02 00:00:00  5        Harbor       26   
1518057    2022-12-14 00:00:00  4        Hollenbeck   35   
2085744    2020-12-29 00:00:00  12       77th Street  12   
2104327    2020-01-08 00:00:00  21       Topanga      28   
3305911    2020-09-04 00:00:00  15       N Hollywood  26   


GENDER  Charge       Disposition_Description      LOCATION                                 LATITUDE  
-----------------------------------------------------------------------------------------------------
M       21310PC      MISDEMEANOR COMPLAINT FILED  WATSON                       AV          33.7753   
M       487(D)(1)PC  MISDEMEANOR COMPLAINT FILED  1500    CRUSADO                      LN  34.058    
M       211PC        MISDEMEANOR COMPLAINT FILED  84TH                                     33.9564   
M       11377HS      MISDEMEANOR COMPLAINT FILED  VENTURA           